### Example for running the RNN

This is an example for running the RNN and get water demand predictions.

Requirements:
- Python 3.6+
- NumPy 1.14+ (http://www.numpy.org/)

To run this Notebook locally:
- Jupyter (https://jupyter.org/)

___Note:___ Current model is an online prediction model. The current version of code does __not__ provide an option for training.

#### Step 1: Import external libraries and the `rnn` package

In [2]:
import os
from rnn import *

#### Step 2: Specify the location on disk of the data directory and the input CSV file (optional)

In [3]:
DATA_LOCATION = os.getcwd()
INPUT_FILE = os.path.join(DATA_LOCATION, 'input_files', 'water_6_17_reduced_2_cap_rs.csv')

#### Step 3: To initialize the predictor RNN, create a `recurrent_neural_network` object.

Required parameters:
- `weights`: Location on disk of the weights file. Typically a Python serialized object (Pickle) file.

Optional parameters:
- `alpha`: Learning rate for the network, typically a value in the range `(0, 1)`. Default value: `0.38`.
- `log`: Boolean value specifying if the execution should be logged to the console. Recommended if running from the command line or in an IDE. Default value: `False`.
- `h_bias`: Boolean value specifying if hidden layer should have a bias node. Currently trained models include a hidden layer bias. Default value: `True`.

In [5]:
rnn = recurrent_neural_network_predict(os.path.join(DATA_LOCATION, 'input_files', 'daily_rnn_retrained_16.pickle'))

At this point, the model is ready to make predictions.

#### Step 4a: Give the network a list of inputs to make predictions

Required parameters:
- `inputs`: A list of `[day_of_the_year, maximum_temperature, precipitation]` lists. These should be real world observed or predicted values.

Optional parameters:
- `networks`: A list of the same length as `inputs` list with values of the range `[0, 1]`, specifying what network each prediction to run on, typically `num_month - 1`. If not specified, all predictions will be made for the month of January.
- `targets`: A list of the same length as `inputs` of the observed values for the day. The target values are required for updating the weights of the model.
- `recal`: Boolean value specifying if the network weights should be recalculated. Recommended if using observed demand as target, to improve future predictions. If set to `True`, requires `targets` to be specified. Default value: `False`.

Output:
- `predictions`: A list of the same length as `inputs` of the predictions made by the network.

In [4]:
inputs = [[1, 71, 0], [2, 101, 121]]
networks = [0, 0]

predictions = rnn.run_network(inputs, networks=networks)

The prediction for inputs  [1, 71, 0]  for network  0  is 110.72
The prediction for inputs  [2, 101, 121]  for network  0  is 110.71


#### Step 4b: Or give the network a list of inputs to make predictions for and targets to learn from

___Note:___ Once retrained, the weights are updated in the object immediately and cannot be undone, unless restored from a previously stored weights file.

In [5]:
inputs = [[1, 71, 0], [2, 101, 121]]
networks = [0, 0]
targets = [106.53, 110.72]

predictions = rnn.run_network(inputs, networks=networks, targets=targets, recal=True)

The target for inputs  [1, 71, 0]  for network  0  was  106.53 and the prediction was 110.72, error observed was 3.93%. 
The target for inputs  [2, 101, 121]  for network  0  was  110.72 and the prediction was 115.86, error observed was 4.64%. 


The network is now retrained and has updated the weights.

(Observe the difference in the second prediction because the weights were updated at the end of the second iteration.)

#### Step 5: Save the predictions to a CSV file

Required parameters:
- `targets`: As above.
- `predictions`: As above.
- `output_file`: Location on disk of the output file, with the name.

In [6]:
write_output_to_file(targets, predictions, os.path.join(DATA_LOCATION, 'predictions_2017.csv'))

#### Step 6: Store the weights for future use

Store the network state for future re-use.

In [7]:
rnn.save(os.path.join(DATA_LOCATION, 'daily_rnn_retrained_17.pickle'))

Further documentation about other functions implemented is available within the source files. The shared variables and parameters are documented in `__init__.py`.

In [7]:
rnn.wih[0]

array([[-1.83495e+00,  3.98117e-01, -2.81294e+00,  7.19808e-01,
        -2.46001e-01,  1.45787e+00,  3.33873e+00,  4.33876e+00],
       [-9.13936e-02, -1.85013e-01, -6.41189e-02,  4.90478e-02,
         5.06962e-02,  6.70162e-02, -3.84283e-01, -2.76644e-01],
       [ 1.04261e-01, -1.69453e-02,  4.78811e-02,  5.13103e-02,
        -8.79883e-04,  2.94587e-02, -1.07619e-02,  5.42553e-02],
       [-1.55184e+00,  3.28430e-01, -2.42274e+00,  5.92234e-01,
        -2.67663e-01,  1.37446e+00,  3.27350e+00,  4.10209e+00]])